In [1]:
import torch
import torchvision 
from torch import nn 
from torchinfo import summary 
from pathlib import Path

from torchvision import transforms


c:\Users\jmhimara\miniconda3\envs\learning\Lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] The specified procedure could not be found'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
image_path = Path("pizza_steak_sushi")

train_dir = image_path / "train"
test_dir = image_path / "test"

In [4]:
import data_setup

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

manual_transform = transforms.Compose([
    transforms.Resize(size=(224,224)), 
    transforms.ToTensor(), 
    normalize
])

train_dl, test_dl, class_names = data_setup.create_dataloaders(train_dir ,test_dir, manual_transform, 32)

In [5]:
weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT

model = torchvision.models.efficientnet_b0(weights = weights).to(device)

In [6]:
# freeze base layers, only reoptimize classifier. 
for param in model.features.parameters():
    param.requires_grad = False 

In [7]:
model.classifier = nn.Sequential(
    nn.Dropout(p=0.2, inplace=True), 
    nn.Linear(1280, out_features=len(class_names))
).to(device)

In [8]:
summary(model, input_size=(32, 3, 224, 224), verbose = 0, 
        col_names = ["input_size", "output_size", "num_params", "trainable"], 
        col_width=20, 
        row_settings=["var_names"])

Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
EfficientNet (EfficientNet)                                  [32, 3, 224, 224]    [32, 3]              --                   Partial
├─Sequential (features)                                      [32, 3, 224, 224]    [32, 1280, 7, 7]     --                   False
│    └─Conv2dNormActivation (0)                              [32, 3, 224, 224]    [32, 32, 112, 112]   --                   False
│    │    └─Conv2d (0)                                       [32, 3, 224, 224]    [32, 32, 112, 112]   (864)                False
│    │    └─BatchNorm2d (1)                                  [32, 32, 112, 112]   [32, 32, 112, 112]   (64)                 False
│    │    └─SiLU (2)                                         [32, 32, 112, 112]   [32, 32, 112, 112]   --                   --
│    └─Sequential (1)                                        [32, 32, 112, 112]   [32, 

In [9]:
model.parameters

<bound method Module.parameters of EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
         

In [10]:
# track results with tensorboard

In [11]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

In [12]:
from torch.utils.tensorboard import SummaryWriter 
writer = SummaryWriter()

In [13]:
writer

In [14]:
from engine import train_step, test_step 
from tqdm.auto import tqdm
from typing import Dict, List, Tuple

def train(model: torch.nn.Module, 
          train_dataloader: torch.utils.data.DataLoader, 
          test_dataloader: torch.utils.data.DataLoader, 
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          epochs: int,
          device: torch.device,
          writer: SummaryWriter) -> Dict[str, List]:
    """Trains and tests a PyTorch model.

    Passes a target PyTorch models through train_step() and test_step()
    functions for a number of epochs, training and testing the model
    in the same epoch loop.

    Calculates, prints and stores evaluation metrics throughout.

    Args:
    model: A PyTorch model to be trained and tested.
    train_dataloader: A DataLoader instance for the model to be trained on.
    test_dataloader: A DataLoader instance for the model to be tested on.
    optimizer: A PyTorch optimizer to help minimize the loss function.
    loss_fn: A PyTorch loss function to calculate loss on both datasets.
    epochs: An integer indicating how many epochs to train for.
    device: A target device to compute on (e.g. "cuda" or "cpu").

    Returns:
    A dictionary of training and testing loss as well as training and
    testing accuracy metrics. Each metric has a value in a list for 
    each epoch.
    In the form: {train_loss: [...],
              train_acc: [...],
              test_loss: [...],
              test_acc: [...]} 
    For example if training for epochs=2: 
             {train_loss: [2.0616, 1.0537],
              train_acc: [0.3945, 0.3945],
              test_loss: [1.2641, 1.5706],
              test_acc: [0.3400, 0.2973]} 
    """
    # Create empty results dictionary
    results = {"train_loss": [],
               "train_acc": [],
               "test_loss": [],
               "test_acc": []
    }
    
    # Make sure model on target device
    model.to(device)

    # Loop through training and testing steps for a number of epochs
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model=model,
                                          dataloader=train_dataloader,
                                          loss_fn=loss_fn,
                                          optimizer=optimizer,
                                          device=device)

        test_loss, test_acc = test_step(model=model,
            dataloader=test_dataloader,
            loss_fn=loss_fn,
            device=device)

        # Print out what's happening
        print(
          f"Epoch: {epoch+1} | "
          f"train_loss: {train_loss:.4f} | "
          f"train_acc: {train_acc:.4f} | "
          f"test_loss: {test_loss:.4f} | "
          f"test_acc: {test_acc:.4f}"
        )

        # Update results dictionary
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)

        # experiment tracking 
        writer.add_scalars(main_tag = "Loss", 
                           tag_scalar_dict = {"train_loss": train_loss, 
                                               "test_loss": test_loss}, 
                            global_step=epoch)

        writer.add_scalars(main_tag = "Accuracy", 
                           tag_scalar_dict = {"train_acc": train_acc, 
                                               "test_acc": test_acc}, 
                            global_step=epoch)
        writer.add_graph(model = model, 
                         input_to_model = torch.randn(32,3,224,224).to(device))
    

    writer.close()
    # Return the filled results at the end of the epochs
    return results


In [15]:
results = train(model, train_dl, test_dl, optimizer, loss_fn, 5, device, writer)

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 1.0589 | train_acc: 0.3867 | test_loss: 0.9469 | test_acc: 0.5994
Epoch: 2 | train_loss: 0.9086 | train_acc: 0.5938 | test_loss: 0.7067 | test_acc: 0.8248
Epoch: 3 | train_loss: 0.7660 | train_acc: 0.8125 | test_loss: 0.6938 | test_acc: 0.7945
Epoch: 4 | train_loss: 0.7512 | train_acc: 0.6836 | test_loss: 0.7350 | test_acc: 0.7235
Epoch: 5 | train_loss: 0.6574 | train_acc: 0.7500 | test_loss: 0.5919 | test_acc: 0.8561


In [71]:
%load_ext tensorboard
%tensorboard --logdir runs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6009 (pid 21000), started 23:33:22 ago. (Use '!kill 21000' to kill it.)

In [60]:
# Write a function to prepare SummaryWriter instance 

# track one experiment per folder. 

def create_writer(experiment_name, 
                  model_name, 
                  extra = None):
    from torch.utils.tensorboard import SummaryWriter
    from datetime import datetime
    import os 

    # get timestamp
    timestamp = datetime.now().strftime("%Y-%m-%d")

    if extra:
        log_dir = os.path.join("runs", timestamp, experiment_name, model_name, extra)
    else:
        log_dir = os.path.join("runs", timestamp, experiment_name, model_name)

    return SummaryWriter(log_dir)


In [33]:
example_writer = create_writer(experiment_name="data_10_percent", 
                               model_name="effnetb0", 
                               extra = "5 epochs")
example_writer

In [34]:
# update train function to include create_writer 

def train(model: torch.nn.Module, 
          train_dataloader: torch.utils.data.DataLoader, 
          test_dataloader: torch.utils.data.DataLoader, 
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          epochs: int,
          device: torch.device,
          writer: SummaryWriter) -> Dict[str, List]:
    """Trains and tests a PyTorch model.
    """
    # Create empty results dictionary
    results = {"train_loss": [],
               "train_acc": [],
               "test_loss": [],
               "test_acc": []
    }
    
    # Make sure model on target device
    model.to(device)

    # Loop through training and testing steps for a number of epochs
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model=model,
                                          dataloader=train_dataloader,
                                          loss_fn=loss_fn,
                                          optimizer=optimizer,
                                          device=device)

        test_loss, test_acc = test_step(model=model,
            dataloader=test_dataloader,
            loss_fn=loss_fn,
            device=device)

        # Print out what's happening
        print(
          f"Epoch: {epoch+1} | "
          f"train_loss: {train_loss:.4f} | "
          f"train_acc: {train_acc:.4f} | "
          f"test_loss: {test_loss:.4f} | "
          f"test_acc: {test_acc:.4f}"
        )

        # Update results dictionary
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)


        if writer:
            # experiment tracking 
            writer.add_scalars(main_tag = "Loss", 
                            tag_scalar_dict = {"train_loss": train_loss, 
                                                "test_loss": test_loss}, 
                                global_step=epoch)

            writer.add_scalars(main_tag = "Accuracy", 
                            tag_scalar_dict = {"train_acc": train_acc, 
                                                "test_acc": test_acc}, 
                                global_step=epoch)
            writer.add_graph(model = model, 
                            input_to_model = torch.randn(32,3,224,224).to(device))
            

    if writer:
        writer.close()

    # Return the filled results at the end of the epochs
    return results


In [63]:
t = [5,6]
t_strings = list(map(lambda x: " ".join(("epoch", str(x))), t))
t_writers = list(map(lambda x: create_writer(x, "model0"), t_strings))

for i,e in enumerate(t):
    train(model = model, 
          train_dataloader=train_dl,
          test_dataloader=test_dl, 
          optimizer=optimizer,
          loss_fn=loss_fn, 
          epochs = e,
          device = device,
          writer = t_writers[i])

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 0.5568 | train_acc: 0.9219 | test_loss: 0.5622 | test_acc: 0.8864
Epoch: 2 | train_loss: 0.5879 | train_acc: 0.7695 | test_loss: 0.5633 | test_acc: 0.8864
Epoch: 3 | train_loss: 0.5357 | train_acc: 0.8125 | test_loss: 0.5524 | test_acc: 0.8968
Epoch: 4 | train_loss: 0.4557 | train_acc: 0.9414 | test_loss: 0.4775 | test_acc: 0.8968
Epoch: 5 | train_loss: 0.4757 | train_acc: 0.8789 | test_loss: 0.4355 | test_acc: 0.8968


  0%|          | 0/6 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 0.5594 | train_acc: 0.7734 | test_loss: 0.4252 | test_acc: 0.8655
Epoch: 2 | train_loss: 0.4179 | train_acc: 0.9453 | test_loss: 0.4405 | test_acc: 0.9176
Epoch: 3 | train_loss: 0.5310 | train_acc: 0.8203 | test_loss: 0.5017 | test_acc: 0.8551
Epoch: 4 | train_loss: 0.4038 | train_acc: 0.9453 | test_loss: 0.3952 | test_acc: 0.9280
Epoch: 5 | train_loss: 0.3779 | train_acc: 0.9531 | test_loss: 0.3983 | test_acc: 0.9280
Epoch: 6 | train_loss: 0.3493 | train_acc: 0.9531 | test_loss: 0.4461 | test_acc: 0.9176
